In [4]:
# リトライ
import random
from retrying import retry

@retry
def proc():
    if random.randint(0, 10) == 5:
        print('process completed')
    else:
        print('retry')
        raise ValueError('invalid value')

proc()

retry
retry
retry
retry
retry
retry
process completed


In [5]:
import random
from retrying import retry

@retry(stop_max_attempt_number=3)
def proc():
    if random.randint(0, 10) == 5:
        print('process completed')
    else:
        print('retry')
        raise ValueError('invalid value')

proc()

retry
retry
retry


ValueError: invalid value

In [22]:
# デコレータ自作
import time

def timed(tar):
    def wrap(*args, **kwargs):
        start = time.time()
        tar(*args, **kwargs)
        print(f'function={tar.__name__}, duration={time.time() - start}')
    return wrap

def logged(tar):
    def wrap(*args, **kwargs):
        print(f'call function: name={tar.__name__}, args={args}, kwargs={kwargs}')
        ret = tar(*args, **kwargs)
        print(f'complete function: name={tar.__name__}, ret={ret}')
        return ret
    return wrap

@timed
@logged
def my_func(p1='hoge', p2='fuga', p3=None):
    print(f'p1={p1}, p2={p2}, p3={p3}')
    time.sleep(0.1)
    return f'p1={p1}, p2={p2}, p3={p3}'

my_func()
print()
my_func('hop', 'step', 'jump')
print()
my_func('', p3='xxx', p2='yyy')

call function: name=my_func, args=(), kwargs={}
p1=hoge, p2=fuga, p3=None
complete function: name=my_func, ret=p1=hoge, p2=fuga, p3=None
function=wrap, duration=0.10062861442565918

call function: name=my_func, args=('hop', 'step', 'jump'), kwargs={}
p1=hop, p2=step, p3=jump
complete function: name=my_func, ret=p1=hop, p2=step, p3=jump
function=wrap, duration=0.1006767749786377

call function: name=my_func, args=('',), kwargs={'p3': 'xxx', 'p2': 'yyy'}
p1=, p2=yyy, p3=xxx
complete function: name=my_func, ret=p1=, p2=yyy, p3=xxx
function=wrap, duration=0.10092997550964355


In [25]:
# 関数名やdocstringの置き換わり
def my_decorator(tar):
    def wrap(*args, **kwargs):
        '''docstring of decorator'''
        tar(*args, **kwargs)
    return wrap

@my_decorator
def my_func(param):
    '''docstring of original'''
    print(param)

print(my_func)
print(my_func.__name__)
print(my_func.__doc__)

<function my_decorator.<locals>.wrap at 0x000002281E50C540>
wrap
docstring of decorator


In [26]:
# functools.wrapsによる問題回避
from functools import wraps

def my_decorator(tar):
    @wraps(tar)
    def wrap(*args, **kwargs):
        '''docstring of decorator'''
        tar(*args, **kwargs)
    return wrap

@my_decorator
def my_func(param):
    '''docstring of original'''
    print(param)

print(my_func)
print(my_func.__name__)
print(my_func.__doc__)

<function my_func at 0x000002281D7A6A20>
my_func
docstring of original
